# **Bài thực hành 2.1: ROI &amp; Lọc nhiễu mặt nạ**

## Tổng quan
Trong bài lab này, bạn sẽ học cách áp dụng vùng quan tâm (ROI) cho mặt nạ làn đường nhị phân và tinh chỉnh mặt nạ bằng các phép toán hình thái học. Đây là bước quan trọng để loại bỏ nhiễu, tập trung vào vùng cần thiết và tạo ra mặt nạ làn đường sạch cho các bước hình học và điều khiển kế tiếp.

## Mục tiêu
- Hiểu khái niệm vùng quan tâm (ROI) trong xử lý ảnh và lý do sử dụng.
- Cài đặt hàm áp dụng đa giác ROI lên mặt nạ nhị phân.
- Hiểu các phép toán hình thái cơ bản như đóng và mở để lọc nhiễu.
- Cài đặt hàm tinh chỉnh mặt nạ bằng cách loại bỏ vùng nhiễu nhỏ và lấp đầy khe hở.

## Yêu cầu nền tảng
Bài lab này giả định bạn đã quen thuộc với Python, NumPy và OpenCV. Bạn cần biết cách đọc ảnh và thực hiện các thao tác mảng đơn giản.

### **Task 1. Cài đặt môi trường**
Trong bài lab này chúng ta sử dụng OpenCV (`cv2`) và NumPy. Chạy ô sau để import các thư viện cần thiết.

khởi tạo các thư viện sẽ sử dụng trong toàn bộ bài lab.
- OpenCV (cv2): dùng để đọc ảnh, xử lý ảnh, tạo ROI và thực hiện các phép toán hình thái học.
- NumPy (np): xử lý mảng số, chuyển đổi kiểu dữ liệu, thao tác trên mask.
- Matplotlib (plt): hiển thị ảnh trước và sau khi áp dụng ROI hoặc refine mask.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

### **Task 2. Tải mặt nạ mẫu**
Trong thực tế, mặt nạ làn đường nhị phân đến từ mô hình phân đoạn bằng mạng nơ-ron. Trong bài lab này, bạn hãy chuẩn bị một ảnh mặt nạ nhị phân (giá trị 0 hoặc 1) và đặt vào thư mục `data/`. Mã dưới đây sẽ load ảnh đó vào mảng NumPy.

##### Khởi tạo đường dẫn input

In [ ]:
# TODO: provide the path to your sample mask image (grayscale PNG or JPG)
mask_path = ...  # e.g. 'data/lane_mask.png' OR r"data\lane_mask.png"

##### Đọc ảnh mask + kiểm tra tồn tại
Mặt nạ lane sau phân đoạn là ảnh nhị phân, chỉ cần 1 kênh duy nhất.
Do đó:
- Không cần đọc ảnh dưới dạng RGB (3 kênh).
- Chỉ cần grayscale để giảm chi phí xử lý và đúng với bản chất dữ liệu.
Ngoài ra, các phép toán ROI và morphology trong pipeline đều giả định mask là mảng 1 kênh.

In [ ]:
# Load the mask image as grayscale
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) 
if mask is None:
    raise FileNotFoundError('Mask file not found. Please place a binary mask image in the data/ folder.')

##### Lưu ý về Chuẩn hóa Mask
Toàn bộ quy trình Mask Processing (bao gồm *ROI Filtering*, *Morphology Closing*, *Connected Components*) đều giả định mask ở dạng nhị phân:

$$mask\_01(x, y) \in \{0, 1\}$$

Tuy nhiên, khi lưu hình ảnh dưới dạng PNG/JPG, các pixel thực tế có thể mang giá trị trong khoảng $0-255$. Do đó, ta cần thực hiện bước chuẩn hóa:

* Pixel $> 0 \to 1$
* Pixel $= 0 \to 0$

> Quan trọng: Đây là bước cần thiết để đảm bảo tính đúng đắn về mặt toán học cho vùng quan tâm (ROI) và giúp các bước tinh chỉnh (refinement) hoạt động chính xác.

In [ ]:
# Convert mask to binary 0/1
mask01 = (mask > 0).astype(np.uint8)

# Display the original mask
plt.figure(figsize=(4, 4))
plt.title('Original mask')
plt.imshow(mask01, cmap='gray')
plt.axis('off')
plt.show()

### **Task 3. Áp dụng vùng quan tâm (ROI)**
##### **Ý tưởng trực quan**

Camera gắn trên AutoCar-Kit có góc nhìn thấp. Trong ảnh:

- Phần trên thường là trần nhà, tường, người, chân bàn…
- Phần dưới mới là mặt đường và vạch làn.

Đối với bài toán giữ làn, chúng ta chỉ cần vùng gần xe – tức là chỉ giữ lại đa giác 40% dưới cùng nơi làn đường thực sự xuất hiện. 
Thay vì xử lý toàn bộ ảnh, ta cắt một vùng ROI (Region of Interest):

> ROI = đa giác bao trùm khu vực mặt đường mà ta quan tâm.

Trong pipeline chính, ROI được định nghĩa bằng các toạ độ tỉ lệ (ratio) so với kích thước ảnh, ví dụ:

##### **Biểu diễn toán học của ROI**

Giả sử ảnh đầu ra của mô hình segmentation có kích thước $(H \times W)$.  
Ta ký hiệu:

- $M(x, y)$: giá trị của mask làn đường tại pixel $(x,y)$, trong đó $M(x,y) \in \{0,1\}$.
- $(x_i^{\text{ratio}},\, y_i^{\text{ratio}})$: tọa độ tỉ lệ của các đỉnh đa giác ROI, trong đó mỗi giá trị thuộc $[0,1]$.

##### **Chuyển tọa độ tỉ lệ → tọa độ pixel**

Để chuyển các điểm ROI về tọa độ pixel thực trong ảnh, ta nhân với chiều cao và chiều rộng:

$$
x_i^{\text{pix}}
= x_i^{\text{ratio}} \cdot W, 
\qquad
y_i^{\text{pix}}
= y_i^{\text{ratio}} \cdot H.
$$

Điều này đảm bảo rằng ROI luôn độc lập với độ phân giải ảnh, và chỉ thay đổi theo kích thước thật $(W, H)$.


##### **Xây dựng mặt nạ ROI**

Ta định nghĩa hàm nhị phân $R(x,y)$ — mặt nạ ROI — như sau:

$$
R(x,y) =
\begin{cases}
1, & \text{nếu } (x,y) \text{ nằm trong đa giác ROI}, \\
0, & \text{nếu } (x,y) \text{ nằm ngoài ROI}. 
\end{cases}
$$

Hàm này được tạo bằng cách tô đầy đa giác ROI trên một ảnh rỗng.


##### **Áp dụng ROI lên mask làn đường**

Mask sau khi áp dụng ROI chính là phép nhân điểm–điểm giữa mask gốc và mặt nạ ROI:

$$
M_{\text{ROI}}(x,y)
= M(x,y) \cdot R(x,y).
$$

Ý nghĩa:

- Nếu $(x,y)$ nằm ngoài ROI  
  ⇒ $R(x,y)=0$  
  ⇒ pixel bị loại bỏ.

- Nếu $(x,y)$ nằm trong ROI 
  ⇒ $R(x,y)=1$  
  ⇒ giá trị mask giữ nguyên.



##### **Minh hoạ**

Áp dụng ROI giúp:

- Loại bỏ toàn bộ phần trên ảnh (nơi không có mặt đường).
- Giảm nhiễu từ vật thể, ánh sáng, bóng người.
- Làm pipeline phía sau (refine mask → BEV → scanline → controller) ổn định hơn.

ROI đóng vai trò như một bộ lọc không gian, chỉ giữ lại những gì xe thực sự quan tâm.


#### **Thực hành – Áp dụng ROI**

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def apply_roi(mask01: np.ndarray, roi_poly: np.ndarray) -> np.ndarray:
    """
    Apply Region of Interest (ROI) mask to the binary lane mask.
    Students will fill in missing code segments.
    """
# -------------------------------------------------------------
# ... YOUR CODE HERE...

    # TODO: Scale ROI polygon to pixel coordinates
    # Hint: use mask01.shape and multiply roi_poly by [W, H]
    H, W = ...          
    pts = ...           
    
    # Create ROI mask and fill polygon with ones
    roi_mask = np.zeros_like(mask01, dtype=np.uint8)
    cv2.fillPoly(roi_mask, [pts], 1)


    # TODO: Zero-out pixels outside ROI
    # Hint: mask_roi should be a copy of mask01 before modification
    mask_roi = ...
    mask_roi[roi_mask == 0] = 0
    return mask_roi


# TODO: Define the ROI polygon (normalized 0–1 coordinates)
# Hint: Use a 4-point polygon (e.g., same as pipeline)
roi_poly = ...

# TODO: Apply ROI using apply_roi(mask01, roi_poly)
mask_roi = ...


# ... END ...
# -------------------------------------------------------------

# Display the result
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.title('Original mask')
plt.imshow(mask01, cmap='gray')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Mask after ROI')
plt.imshow(mask_roi, cmap='gray')
plt.axis('off')

plt.show()


### **Task 4. Tinh chỉnh mặt nạ**

Sau khi áp dụng ROI, mask làn đường vẫn thường chứa nhiều lỗi do mô hình dự đoán:

- Làn bị đứt đoạn (gaps),
- Xuất hiện đốm trắng nhỏ (noise),
- Có nhiều vùng rời rạc không phải làn thật.

Mục tiêu của bước mask refinement là:

> Biến mask thô ban đầu thành một vùng làn duy nhất – sạch – liên tục – ổn định cho bước hình học (geometry).

Để làm điều đó, pipeline thực hiện 3 thao tác chính:
1. Chuẩn hóa mask về dạng nhị phân {0,1}  
2. Áp dụng morphological closing để nối liền làn đường  
3. Giữ lại vùng (contour) có diện tích lớn nhất


##### **Chuẩn hóa mask về dạng nhị phân**

Trong nhiều trường hợp, mô hình có thể trả về mask với các giá trị khác nhau (0 hoặc >0).  
Ta quy về dạng nhị phân:

$$
m(x,y) =
\begin{cases}
1, & \text{khi } M(x,y) > 0, \\
0, & \text{ngược lại}.
\end{cases}
$$

Điều này giúp đảm bảo mask luôn là dữ liệu "sạch" trước khi thực hiện các phép toán hình thái học.


##### **Morphological Closing – Nối các đoạn làn bị đứt**

#####  Định nghĩa toán học

Phép closing của ảnh nhị phân $(m)$ với kernel $(K)$ là:

$$
m \bullet K = (m \oplus K) \ominus K
$$

Trong đó:

- $(m \oplus K)$: dilation – giãn vùng trắng (kết nối các điểm gần nhau)  
- $(m \ominus K)$: erosion – co vùng trắng lại (loại bỏ phình ra do dilation)

Ý nghĩa trực quan:

> Dilation lấp các khoảng trống nhỏ → Erosion thu lại → kết quả giữ nguyên đường nét lớn nhưng không còn khe hở nhỏ.


#####  Kernel dọc (5 × 25)

Pipeline sử dụng kernel:

- Rộng 5 pixel → không làm hai làn kế bên bị dính vào nhau  
- Cao 25 pixel → đủ dài để nối các khoảng đứt đoạn theo hướng dọc (hướng lane)

Kernel:

```python
ker_vertical = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 25))


#### **Thực hành – Lọc nhiễu**

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def refine_mask01(mask01: np.ndarray) -> np.ndarray:
    """
    Refine the binary lane mask by applying morphological closing
    and removing small connected components.
    Students will fill in missing code segments.
    """
# -------------------------------------------------------------
# ...YOUR CODE HERE...

    # TODO: Convert (0/1) mask into uint8 0–255
    mask_uint8 = ...

    # -------------------------------------------------------------
    # TODO: Apply morphological closing with a vertical kernel (5×25)
    kernel = ...
    closed = ...

    # -------------------------------------------------------------
    # TODO: Connected components: keep only the largest 1–2 components
    num_labels, labels, stats, centroids = ...
    areas = ...
    sorted_idx = ...
    mask_filtered = ...
    # Hint: loop over sorted_idx[:2] and assign 255 to the largest components
    
    # -------------------------------------------------------------
    # TODO: Convert refined mask back to binary (0/1)
    refined = ...
    return refined


# -------------------------------------------------------------
# TODO: Apply refine_mask01() to the ROI mask
refined_mask = ...


# ... END ...
# -------------------------------------------------------------


# Visualization
plt.figure(figsize=(8, 4))

plt.subplot(1, 2, 1)
plt.title('Mask after ROI')
plt.imshow(mask_roi, cmap='gray')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Refined mask')
plt.imshow(refined_mask, cmap='gray')
plt.axis('off')

plt.tight_layout()
plt.show()


### **Task 5. Lưu ảnh**


In [ ]:
import os
import cv2
import time

# mask_refined: the final refined mask (binary 0/1 or 0/255)

# 1) Create the folder result_refine if it does not already exist
save_dir = "result_refine"
os.makedirs(save_dir, exist_ok=True)

# 2) Generate an output filename using a timestamp to avoid overwriting
filename = f"refined_{int(time.time())}.png"
save_path = os.path.join(save_dir, filename)

# 3) Convert the mask to 0–255 format before saving
mask_to_save = (refined_mask * 255).astype("uint8")

# 4) Save the image
cv2.imwrite(save_path, mask_to_save)

print("Refined mask has been saved at:", save_path)


### Tổng kết
Trong bài lab này, bạn đã triển khai hai bước tiền xử lý quan trọng cho mặt nạ phân đoạn làn đường: áp dụng vùng quan tâm và tinh chỉnh mặt nạ bằng các phép toán hình thái. Mặt nạ sạch là cơ sở để tính toán hình học chính xác và điều khiển làn đường ổn định.